In [1]:
%pylab inline
import sys
sys.path.append('/Users/ggarcia/Documents/git_sources/sdg/cmake-build-relwithdebinfo/SDGpython/')
import SDGpython as SDG
from collections import Counter

Populating the interactive namespace from numpy and matplotlib


In [2]:
## SDG-DBG assembly k=63
## Min coverage is important to reduce the noice
K=63
MIN_CVG=10
NUM_BATCHES=1
ws=SDG.WorkSpace()
peds=ws.add_paired_reads_datastore('./pseudo_triploid_octoploid_new/octoploid_pe.prseq', 'pe')

gm=SDG.GraphMaker(ws.sdg)
gm.new_graph_from_paired_datastore(peds,K,MIN_CVG,NUM_BATCHES);

gc=SDG.GraphContigger(ws)
gc.clip_tips(2*K-1,3)
print("Remaining nodes: %s" %(len(ws.sdg.get_all_nodeviews())))

ws.sdg.write_to_gfa1("./sdg_dbg_graph.gfa")

kc = ws.add_kmer_counter('pek31', 31)
kc.add_count('pe', peds)

## Make unique kmer runs
seqs = gc.contig_reduction_to_unique_kmers(60, 100)            
print(len(seqs),"sequences for anchoring")
with open("pseudoLong2.fastq","w") as f:
    for s in seqs:
        f.write("@seq\n%s\n+\n%s\n"%(s,s))

Remaining nodes: 23705
10391 sequences for anchoring


In [3]:
## Recreate workspace using only unique kmer runs
ws=SDG.WorkSpace()
peds=ws.add_paired_reads_datastore('./pseudo_triploid_octoploid_new/octoploid_pe.prseq')
lords=ws.add_long_reads_datastore('./pseudo_triploid_octoploid_new/octoploid_long.loseq')
for s in seqs:
    ws.sdg.add_node(s)
print(len(ws.sdg.get_all_nodeviews()))
kc = ws.add_kmer_counter('pek31', 31)
kc.add_count('pe', peds)

lords = ws.add_long_reads_datastore('./pseudo_triploid_octoploid_new/octoploid_long.loseq')
lrr=SDG.LongReadsRecruiter(ws.sdg,lords,31)
lrr.map(31)
lrr.thread_reads(end_matches=1)
# tdg=lrr.dg_from_threads()
peds.mapper.path_reads(31)

10391


In [4]:
## Group nodes using the next neighbour and pe
def make_neighbours_graph():
    sedg=SDG.DistanceGraph(ws.sdg)
    pedg=SDG.DistanceGraph(ws.sdg) # only links when there is no SE next.
    
    for nv in ws.sdg.get_all_nodeviews(both_directions=True):
#         print(nv.node_id())
        for p in peds.mapper.all_paths_fw(nv.node_id()):
            if len(p)==0: continue
            if p[0]!=0:
                sedg.add_link(-nv.node_id(), p[0], 0)
            else:
                pedg.add_link(-nv.node_id(), p[1], 0)

    return sedg, pedg

sedg, pedg = make_neighbours_graph()

sedg.write_to_gfa1("./neghbour_sedg.gfa")
pedg.write_to_gfa1("./neghbour_pedg.gfa")

In [5]:
## Remove multiple conections by voting the most popular
def pick_next(nv, min_votes=10):
    """ Min votes to be the poll winner """
    winner = None
    poll = Counter([x.node().node_id() for x in nv.next()]).most_common(1)
    if len(poll)>0 and poll[0][1]>min_votes:
        winner = poll[0][0]
    return winner

def clean_conections(idg):
    """ From the same read neighbour graph multilinkage graph (sedg) picks the most popular link as winner """
    ndg = SDG.DistanceGraph(ws.sdg)
    for nv in idg.get_all_nodeviews(both_directions=True):
        winner = pick_next(nv)
        if winner is not None:
            if not ndg.are_connected(-nv.node_id(), winner):
                ndg.add_link(-nv.node_id(), winner, 0)
                
    ## Disconect multiple connections to create the lines that are going to be the groups
    for nv in ndg.get_all_nodeviews(both_directions=True):
        if len(nv.next())>1:
            for con in nv.next():
                ndg.remove_link(-nv.node_id(), con.node().node_id())

    return ndg

dg = clean_conections(sedg)
dg2 = clean_conections(pedg)

dg.write_to_gfa1("./neghbour_dg.gfa")
dg2.write_to_gfa1("./neghbour_dg2.gfa")

In [6]:
## group components
def get_components(dg):
    """ Transform lines to groups """
    all_nodes = set([n.node_id() for n in dg.get_all_nodeviews()])
    groups={}
    while len(all_nodes)>0:
        node = all_nodes.pop()
        group = [node, ]
        fwn = dg.get_nodeview(node).next()
        while len(fwn)==1:
            next_node = fwn[0].node().node_id()
            if next_node in group: break
            group.append(next_node)
            all_nodes.remove(abs(next_node))
            fwn = dg.get_nodeview(next_node).next()


        group = [x*-1 for x in group[::-1]]
        fwn = dg.get_nodeview(-node).next()
        while len(fwn)==1:
            next_node = fwn[0].node().node_id()    
            if next_node in group: break                
            group.append(next_node)
            
            all_nodes.remove(abs(next_node))
            fwn = dg.get_nodeview(next_node).next()
        groups[node] = group    
    return groups

groups = get_components(dg)

In [7]:
## transformation table from nodo to group
print(len(groups))
transformation = {}
for nodo_central, integrantes in groups.items():
    for integrante in integrantes:
        transformation[integrante] = nodo_central
        transformation[-integrante] = -nodo_central

2988


In [8]:
## transform reads to group mappings
trpm=[]
for rpm in lrr.read_perfect_matches:
    if len(rpm)<2: 
        trpm.append([])
        continue
    ms=[transformation[rpm[0].node]]
    for ix in range(1,len(rpm)-1):
        x=rpm[ix]
        g=transformation[x.node]
        if g==ms[-1]: continue #same group as before
        if ms[-1]==transformation[rpm[ix+1].node]: continue #sandwich
        ms.append(g)
    if transformation[rpm[-1].node]!=ms[-1]: ms.append(transformation[rpm[-1].node])
    trpm.append(ms)

In [9]:
def gid_reads():
    """ collection to enter using the group id in absolute value and return the mapped reads to translate later"""
    gid2reads={abs(gid):set() for gid in groups.keys()}
    for i, read in enumerate(trpm):
        for gid in read:
            gid2reads[abs(gid)].add(i)
    return gid2reads

gid2reads = gid_reads()


def get_next(gid):
    """ Given a group id return the next group using long reads """
    c=Counter()
    trpm_ids = gid2reads[abs(gid)]
    ttrpm = [trpm[i] for i in trpm_ids]
    for x in ttrpm:
        if -gid in x: 
            x= [y*-1 for y in x[::-1]]
        if gid in x and x[-1] != gid:
            c[x[x.index(gid)+1]]+=1
    if len(c.most_common())>0 and c.most_common(1)[0][1]>3:
        return c.most_common(1)[0][0]
    else:
        None
        

In [10]:
ws2 = SDG.WorkSpace()
## Add all groups as nodes
group2node={}
node2group={}
all_nodes = [nodo for k, group in groups.items() for nodo in group]
for gid in groups.keys():
    group_size = sum([ws.sdg.get_nodeview(x).size() for x in groups[gid]])
    new_node = ws2.sdg.add_node("A"*group_size) 
    group2node[gid] = new_node
    node2group[new_node] = gid

print("%s nodes in ws2 graph" %len(ws2.sdg.get_all_nodeviews()))

# def runall():
ndg = SDG.DistanceGraph(ws2.sdg)
for i, g in enumerate(groups.keys()):
    visited=set()
    prev_group = None
    next_group = get_next(g)
    while next_group is not None and next_group not in visited:
        if prev_group is not None:
            if prev_group<0: n1=-group2node[abs(prev_group)]
            else: n1=group2node[abs(prev_group)]

            if next_group<0: n2=-group2node[abs(next_group)]
            else: n2=group2node[abs(next_group)]

#                 print("%s --> %s" %(n1, n2))
#             if not ws.sdg.are_connected(-n1, n2):
            ndg.add_link(-n1, n2, 0, SDG.Support())
            

        visited.add(next_group)
        prev_group = next_group
        next_group = get_next(next_group)
    print("\r%s/%s" %(i, len(groups)), end='')
#         if i%50==1:
#             ws2.sdg.write_to_gfa1('./dummygraph.gfa')


2988 nodes in ws2 graph
2987/2988

In [11]:
ndg.write_to_gfa1('./dummygraph.gfa')

In [12]:
u=SDG.LinkageUntangler(ndg)
u.select_all()
print(len([1 for x in u.selected_nodes if x==True]),"nodes selected to start with")
nsl=u.make_nextselected_linkage(7)
nsl.remove_transitive_links()

2989 nodes selected to start with


In [13]:
nsl.write_to_gfa1("./nsl_graph.gfa")

In [14]:
## deduplicate sdg connections 

In [15]:
for nv in nsl.get_all_nodeviews():
    ## Bring group
    group = node2group[nv.node_id()]
    group_nodes = groups[group]
    print("____")
    print("Node id: %s, gorup id: %s, nodes in group %s" %(nv.node_id(), group, group_nodes))
    print("Node %s prevs %s" %(group_nodes[0], Counter([x.node().node_id() for x in sedg.get_nodeview(group_nodes[0]).prev()]).most_common()))
    print("Node %s nexts %s" %(group_nodes[-1], Counter([x.node().node_id() for x in sedg.get_nodeview(group_nodes[-1]).next()]).most_common()))

    print("Prevs")        
    for pn in nv.prev():
        prev_group = node2group[abs(pn.node().node_id())]
        prev_group_nodes = groups[prev_group]       
        print(pn.node().node_id(), prev_group, prev_group_nodes)#, prev_group_nodes[0], prev_group_nodes[-1])
        
    print("Nexts")
    for nn in nv.next():
        next_group = node2group[abs(nn.node().node_id())]
        next_group_nodes = groups[next_group]
        print(nn.node().node_id(), next_group, next_group_nodes)#, next_group_nodes[0], next_group_nodes[-1])
        

        
    ## bring next group
    ## Orient

____
Node id: 1, gorup id: 1, nodes in group [-8413, -1]
Node -8413 prevs [(6384, 2)]
Node -1 nexts [(-6248, 10), (4077, 2)]
Prevs
827 1417 [-7582, 7581, -1417, 8914, -8933, -7880, 4045]
Nexts
513 837 [-6248, 4021, 4685, -837]
____
Node id: 2, gorup id: 2, nodes in group [-5008, -2, -3]
Node -5008 prevs [(2354, 10)]
Node -3 nexts [(5534, 2)]
Prevs
-1248 2354 [-2354, 2355, 9652, 5230, 8263]
Nexts
-1563 3210 [-3210, 5532, 5533, -5534]
____
Node id: 3, gorup id: 4, nodes in group [-4, 7488, 7487, 5739]
Node -4 prevs []
Node 5739 nexts []
Prevs
-1814 4002 [-4002]
Nexts
-2822 9218 [-9218]
____
Node id: 4, gorup id: 5, nodes in group [-183, -182, -5]
Node -183 prevs [(-9909, 2), (-107, 2)]
Node -5 nexts [(-7062, 2), (-5558, 2)]
Prevs
-579 959 [4482, 9459, 8257, -6102, -6367, -959]
Nexts
-2504 7117 [9711, 7118, -7117]
____
Node id: 5, gorup id: 6, nodes in group [-6, -7, 4332]
Node -6 prevs []
Node 4332 nexts [(-3779, 2), (9979, 2)]
Prevs
1455 2841 [-8558, -2841, -2842]
Nexts
1927 4413 [-4413

Node 4410 nexts []
Prevs
-1255 2369 [-2369]
Nexts
-2549 7334 [-7334]
____
Node id: 98, gorup id: 142, nodes in group [-143, -142]
Node -143 prevs [(-596, 2), (1255, 2)]
Node -142 nexts []
Prevs
2852 9406 [-9406]
Nexts
2442 6694 [-6694, 10142, 8935, 8934]
____
Node id: 99, gorup id: 144, nodes in group [882, 881, -8447, -144, 5514, -5513, 9607]
Node 882 prevs []
Node 9607 nexts [(9621, 2)]
Prevs
360 572 [-4308, 640, -572, 1872, 2329, -8268, 8269, 8270]
Nexts
398 635 [-635]
____
Node id: 100, gorup id: 145, nodes in group [-381, 380, -146, -145, 9526]
Node -381 prevs []
Node 9526 nexts [(-7420, 2)]
Prevs
-1040 1866 [-1866, -6192]
Nexts
1187 2181 [-2181, 7908, -7907, -7906]
____
Node id: 101, gorup id: 147, nodes in group [-4166, -951, -950, 4504, -147, -1205]
Node -4166 prevs [(-5009, 2)]
Node -1205 nexts [(-2736, 2)]
Prevs
682 1138 [-1138, 1512]
Nexts
2008 4761 [4762, -4761, 7540]
____
Node id: 102, gorup id: 148, nodes in group [6251, 1794, -3543, -148]
Node 6251 prevs []
Node -148 nex

-160 237 [8167, 7164, -7165, 5503, -237, -2457]
____
Node id: 139, gorup id: 206, nodes in group [-207, -206]
Node -207 prevs []
Node -206 nexts []
Prevs
-2167 5396 [-5396]
Nexts
1639 3429 [-3429, 9109, -9108]
____
Node id: 140, gorup id: 208, nodes in group [-208]
Node -208 prevs [(5771, 2), (6864, 2)]
Node -208 nexts []
Prevs
1888 4261 [-4261]
Nexts
517 841 [-4251, 4619, -4618, 7760, 7759, 3979, 3978, 5917, -841, 842, -843, -844, 845, 846]
____
Node id: 141, gorup id: 209, nodes in group [-5923, -5924, -614, -613, -612, 4118, 8008, -8007, 8006, -6970, 6969, 6968, -6967, -210, -209]
Node -5923 prevs []
Node -209 nexts []
Prevs
-598 995 [-4669, -4668, -995]
Nexts
-539 883 [-2506, 900, -884, -883, -7718, 7719]
____
Node id: 142, gorup id: 211, nodes in group [-4117, 2875, -2874, 3993, 3994, 3995, 9742, -212, -211]
Node -4117 prevs [(-611, 4)]
Node -211 nexts []
Prevs
387 611 [-611]
Nexts
-748 1273 [-1273, 1274, 2384, -2385, -1758, 1759, -7288]
____
Node id: 143, gorup id: 213, nodes in 

Node -5415 prevs []
Node -340 nexts []
Prevs
2539 7280 [-7911, 10220, -7280]
Nexts
1796 3958 [-10048, 10049, 8683, -3958]
____
Node id: 223, gorup id: 341, nodes in group [-394, -6746, -9470, 9469, -9468, 343, 342, -341, 3133]
Node -394 prevs [(-4572, 4), (-45, 2)]
Node 3133 nexts [(8824, 8), (-8823, 2), (-7407, 2), (7726, 2)]
Prevs
-76 105 [-105, 106, -107]
Nexts
2775 8823 [8824, -8823]
____
Node id: 224, gorup id: 344, nodes in group [9299, 4059, -344, 6610, -10251, -10250, -1655, -1656, -8991, -2395, 3320, 6100]
Node 9299 prevs []
Node 6100 nexts [(7964, 6)]
Prevs
-2165 5388 [-5389, -5388]
Nexts
1259 2376 [-2376]
____
Node id: 225, gorup id: 345, nodes in group [2876, -345]
Node 2876 prevs [(-1935, 2), (-1382, 2), (590, 2)]
Node -345 nexts [(-8434, 2), (-3952, 2), (1381, 2), (2730, 2)]
Prevs
-489 794 [-4214, 4364, -794, -7175]
Nexts
1612 3362 [-9974, -9975, 9190, -4803, -4804, -4805, -4806, -4807, -8875, 8874, -3362]
____
Node id: 226, gorup id: 346, nodes in group [-346, -8422, -83

Node 423 nexts [(-8406, 6)]
Prevs
2680 8211 [-9906, 9665, -8211]
Nexts
2702 8404 [-8406, 8405, -8404]
____
Node id: 265, gorup id: 424, nodes in group [-9878, -5491, -424, -3348]
Node -9878 prevs [(6616, 2)]
Node -3348 nexts []
Prevs
-2690 8307 [8697, -8307]
Nexts
2638 7899 [-10381, -10380, -7899, -9881]
____
Node id: 266, gorup id: 425, nodes in group [-4036, 4037, -425, 6437, -649, -648, 647]
Node -4036 prevs []
Node 647 nexts [(-8808, 2), (8095, 2)]
Prevs
2034 4837 [-4837, 4838, 8296]
Nexts
455 746 [-5521, -5522, -5523, 9755, -746]
____
Node id: 267, gorup id: 426, nodes in group [5973, -426, 1190]
Node 5973 prevs [(-4011, 2)]
Node 1190 nexts []
Prevs
840 1442 [-1442, 6548, 7858, -7859]
Nexts
-1631 3407 [-3407, 3408, -5697, -6776, 10009]
____
Node id: 268, gorup id: 428, nodes in group [-428, 429]
Node -428 prevs [(-1427, 2)]
Node 429 nexts [(-1475, 2)]
Prevs
-230 357 [1959, -357, 2885, 2884, 2883]
Nexts
2591 7573 [-7573]
____
Node id: 269, gorup id: 430, nodes in group [-6740, -674

...

Node -6810 nexts []
Prevs
-2582 7530 [-7530]
Nexts
644 1079 [-10232, -1079]
____
Node id: 2459, gorup id: 6811, nodes in group [-6811]
Node -6811 prevs [(9798, 2)]
Node -6811 nexts []
Prevs
632 1060 [4139, -4138, -4137, -4136, 7713, -1060]
Nexts
-1917 4362 [4363, -4362, 9096, -9095, -4607, -4606]
____
Node id: 2460, gorup id: 6830, nodes in group [-8126, 8125, -6830, 6831]
Node -8126 prevs [(-3971, 2)]
Node 6831 nexts []
Prevs
2081 5037 [8021, 6242, -5037, 5309, -8656, 8657]
Nexts
506 823 [-823, 6270, -6271, 3884]
____
Node id: 2461, gorup id: 6834, nodes in group [-6837, 6836, 6835, -6834]
Node -6837 prevs [(-5988, 2), (6838, 2)]
Node -6834 nexts []
Prevs
-2956 10145 [-10331, 10148, 10147, -10146, -10145]
Nexts
-2843 9352 [-9353, -9352]
____
Node id: 2462, gorup id: 6839, nodes in group [-6839]
Node -6839 prevs [(3311, 2)]
Node -6839 nexts [(1025, 2), (3157, 2)]
Prevs
-934 1636 [-2821, -2820, -2533, 1637, -1636, 4174, 4173, -4172, -4171]
Nexts
-617 1025 [-1025]
____
Node id: 2463, gor

460 753 [-9268, 5691, -753, 7715]
____
Node id: 2576, gorup id: 7508, nodes in group [-10061, 10062, 7509, -7508]
Node -10061 prevs []
Node -7508 nexts [(-2749, 2)]
Prevs
-443 712 [-712]
Nexts
-674 1124 [-10235, 1815, 4001, -4000, -2095, 2096, -1124, -1125, -9217, -9216, -9215]
____
Node id: 2577, gorup id: 7510, nodes in group [-7510]
Node -7510 prevs [(-2957, 2)]
Node -7510 nexts []
Prevs
1695 3601 [-3601, 5330, -6739, 7533]
Nexts
-2500 7098 [-7098]
____
Node id: 2578, gorup id: 7520, nodes in group [-7520]
Node -7520 prevs []
Node -7520 nexts [(-4529, 2)]
Prevs
2238 5690 [7204, 5931, -5690, -8761]
Nexts
-298 479 [-7963, 7180, -5254, -479]
____
Node id: 2579, gorup id: 7523, nodes in group [-7523, -7524]
Node -7523 prevs []
Node -7524 nexts []
Prevs
-838 1440 [-1440]
Nexts
-2147 5305 [-5305]
____
Node id: 2580, gorup id: 7525, nodes in group [-7525]
Node -7525 prevs [(-4020, 4)]
Node -7525 nexts [(-4279, 2)]
Prevs
-2003 4722 [-7924, -4722]
Nexts
-200 304 [-7650, 7651, 7652, 305, -304

Node id: 2696, gorup id: 8363, nodes in group [-8363]
Node -8363 prevs []
Node -8363 nexts [(-10019, 2), (-8773, 2), (-8378, 2), (-14, 2)]
Prevs
-862 1486 [-1486, 7535]
Nexts
-1558 3176 [7974, -3176]
____
Node id: 2697, gorup id: 8375, nodes in group [-8378, -8377, -8376, -8375]
Node -8378 prevs [(-10021, 2), (-8363, 2)]
Node -8375 nexts []
Prevs
845 1452 [-5515, -1452, -1453]
Nexts
1411 2743 [-2743]
____
Node id: 2698, gorup id: 8379, nodes in group [-8379]
Node -8379 prevs []
Node -8379 nexts []
Prevs
Nexts
____
Node id: 2699, gorup id: 8384, nodes in group [-8384]
Node -8384 prevs []
Node -8384 nexts []
Prevs
-2792 8942 [-8942]
Nexts
-1598 3319 [-3319, 4437]
____
Node id: 2700, gorup id: 8387, nodes in group [9194, -9193, -8387, -8401, 8402]
Node 9194 prevs []
Node 8402 nexts [(-2616, 2)]
Prevs
-1553 3157 [-3157, 3158, -3159, -6006, -7224, -8189, 8229]
Nexts
-184 277 [-277, -1659, 1660, 8660, 497]
____
Node id: 2701, gorup id: 8403, nodes in group [-8403, 9753]
Node -8403 prevs [(-4

795 1354 [3733, -1354, 1355, -1356]
Nexts
2889 9686 [-9686]
____
Node id: 2787, gorup id: 8925, nodes in group [-8925]
Node -8925 prevs []
Node -8925 nexts []
Prevs
-1000 1790 [-1790, 3469, 3470]
Nexts
1517 3051 [-3051]
____
Node id: 2788, gorup id: 8928, nodes in group [-8928]
Node -8928 prevs []
Node -8928 nexts [(-9400, 2)]
Prevs
-1609 3358 [-3358, -4404]
Nexts
29 39 [9260, 9259, -39, -2079]
____
Node id: 2789, gorup id: 8932, nodes in group [-10336, 10335, -8932]
Node -10336 prevs [(-2938, 2)]
Node -8932 nexts [(8468, 4)]
Prevs
2109 5166 [-6646, -6647, -5166, -5167]
Nexts
834 1432 [8468, 1433, -1432, 7914, -8675, 8676, -2056, 9732, 9733, 9734, -9093, 9119, -5321]
____
Node id: 2790, gorup id: 8937, nodes in group [-8937]
Node -8937 prevs []
Node -8937 nexts []
Prevs
-734 1247 [-1247, 1248, -1249, -7621, 4873]
Nexts
125 189 [4090, -189, -7399, -4941, -8164, -8163, 8162, 8161, -8160]
____
Node id: 2791, gorup id: 8938, nodes in group [8940, 8939, -8938]
Node 8940 prevs [(-10389, 2), 

-2171 5405 [-5405, 5569, 5588]
Nexts
-1820 4022 [-4022]
____
Node id: 2902, gorup id: 9759, nodes in group [-9759]
Node -9759 prevs [(-369, 2), (6622, 2)]
Node -9759 nexts []
Prevs
-1566 3222 [9864, 7011, -3223, -3222, -4610]
Nexts
-1632 3409 [-3409, 3410]
____
Node id: 2903, gorup id: 9765, nodes in group [-9765]
Node -9765 prevs [(-7473, 2)]
Node -9765 nexts []
Prevs
269 430 [-6740, -6741, -430, -4809, 9223, -1522, 1521]
Nexts
-2375 6293 [-6294, -6293]
____
Node id: 2904, gorup id: 9768, nodes in group [-9768]
Node -9768 prevs [(-698, 2)]
Node -9768 nexts []
Prevs
-2653 8017 [-8019, 8018, -8017]
Nexts
1897 4288 [-4288]
____
Node id: 2905, gorup id: 9770, nodes in group [-9770]
Node -9770 prevs []
Node -9770 nexts [(-6580, 2), (-2457, 2), (2456, 2)]
Prevs
-332 531 [-531, 6216, -1881, -7826, -7827]
Nexts
2423 6580 [-6580]
____
Node id: 2906, gorup id: 9775, nodes in group [-9775]
Node -9775 prevs []
Node -9775 nexts [(-8482, 2)]
Prevs
497 809 [8435, -8436, 810, -809, -9178, -9177, 9176

In [ ]:
def connected_components(graph):
    visited = []
    remaining = [nv for nv in graph.get_all_nodeviews()]
    to_visit = [remaining.pop(), ]
    
    while len(to_visit)>0:
        print("\r%s %s" %(len(to_visit), len(visited)), end="")
        node = to_visit.pop()
        if node in visited: continue
        visited.append(node)  
        for lv in node.next():
            if lv.node() not in to_visit:
                to_visit.append(lv.node())
                
        for lv in node.prev():
            if lv.node() not in to_visit:
                to_visit.append(lv.node())
        
    return visited
# componentes = connected_components(ws2.sdg)
# print(len(componentes))

In [ ]:
print(len(componentes))

In [16]:
print(ws.ls())
peds = ws.get_paired_reads_datastore('')

SDG Workspace
  SequenceDistanceGraph SDG: 10391 nodes, 0 links
  Distance Graphs: 0
  Paired Read Datastores: 1
    Paired Reads Datastore octoploid_pe: 736696 reads
      Paired Reads Mapper: 0 mapped reads, 0 unmapped
  Linked Read Datastores: 0
  Long Read Datastores: 2
    Long Reads Datastore octoploid_long: 5525 reads
      Long Reads Mapper: 0 raw mappings
    Long Reads Datastore octoploid_long: 5525 reads
      Long Reads Mapper: 0 raw mappings
  Kmer Count Datastores: 1
    KmerCounter pek31: index with 224650 31-mers
      Counts: 2
        sdg: 224678 total 31-mers
        pe: 16089131 total 31-mers

